<a href="https://colab.research.google.com/github/Zhangylay-2000/de-zoomcamp/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Homework_data_talks_club_data_extraction_and_ingestion_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
from re import L
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# 1
limit = 5
generator = square_root_generator(limit)
sum_of_outputs = sum(generator)
print("Sum of the outputs where the limit is 5:", sum_of_outputs)

#2
limit2=13
gen = square_root_generator(limit2)

for _ in range(limit2-1):
  next(gen)
thirteen_n =next(gen)
print("13th number yielded:",thirteen_n)


Sum of the outputs where the limit is 5: 8.382332347441762
13th number yielded: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [10]:
import duckdb

# Define the first generator
def people_1():
    for i in range(1, 6):
        yield (i, f"Person_{i}", 25 + i, "City_A", None)

# Define the second generator
def people_2():
    for i in range(3, 9):
        yield (i, f"Person_{i}", 30 + i, "City_B", f"Job_{i}")

# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Create a table
con.execute("""
CREATE TABLE People4 (
    ID INTEGER,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
""")

# Load data from the first generator into the table
gen_1 = people_1()
for record in gen_1:
    con.execute("INSERT INTO People4 VALUES (?, ?, ?, ?, ?)", record)

# Append data from the second generator to the table
gen_2 = people_2()
for record in gen_2:
    con.execute("INSERT INTO People4 VALUES (?, ?, ?, ?, ?)", record)

# Calculate the sum of ages of all people
sum_of_ages = con.execute("SELECT SUM(Age) FROM People4").fetchone()[0]
print("Sum of ages of all people:", sum_of_ages)


Sum of ages of all people: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [11]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [13]:
import duckdb

# Define the first generator
def people_1():
    for i in range(1, 6):
        yield (i, f"Person_{i}", 25 + i, "City_A", None)

# Define the second generator
def people_2():
    for i in range(3, 9):
        yield (i, f"Person_{i}", 30 + i, "City_B", f"Job_{i}")

# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Create a table with primary key ID
con.execute("""
CREATE TABLE People6 (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
""")

# Load data from the first generator into the table
gen_1 = people_1()
for record in gen_1:
    con.execute("INSERT INTO People6 VALUES (?, ?, ?, ?, ?)", record)

# Use MERGE to load data from the second generator, replacing records with overlapping IDs
gen_2 = people_2()
for record in gen_2:
    con.execute("INSERT OR REPLACE INTO People6 VALUES (?, ?, ?, ?, ?)", record)

# Calculate the sum of ages of all people
sum_of_ages = con.execute("SELECT SUM(Age) FROM People6").fetchone()[0]
print("Sum of ages of all people:", sum_of_ages)


Sum of ages of all people: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX